In [1]:
import pandas as pd

# Converts CEN scores to distance scores.

In [2]:
df = pd.read_csv('../data/roberta/Astro1_decomposed_clean_pairs_roberta.csv')
pairs_df = pd.read_csv('../data/Astro1_decomposed_clean_pairs.csv')
display(pairs_df.head(), df.head())

,Unnamed: 0,first,second
0,0,You never give up.,You never find it difficult to change your min...
1,1,You never give up.,"Whatever you have set your sights on, you refu..."
2,2,You never give up.,You are patient unless someone takes you too far.
3,3,You never give up.,You are usually slow to anger unless someone t...
4,4,You never give up.,You are reliable.


,Unnamed: 0,predicted_class,C,N,E
0,0,neutral,0.022798,0.880442,0.096760
1,1,entailment,0.001746,0.039801,0.958454
2,2,neutral,0.004882,0.990273,0.004845
3,3,neutral,0.052050,0.944104,0.003846
4,4,entailment,0.001493,0.161846,0.836661


In [3]:
df = df.drop(columns=['Unnamed: 0'])
pairs_df = pairs_df.drop(columns=['Unnamed: 0'])
display(df, pairs_df)

,predicted_class,C,N,E
0,neutral,0.022798,0.880442,0.096760
1,entailment,0.001746,0.039801,0.958454
2,neutral,0.004882,0.990273,0.004845
3,neutral,0.052050,0.944104,0.003846
4,entailment,0.001493,0.161846,0.836661
...,...,...,...,...
1076,entailment,0.042239,0.253214,0.704547
1077,contradiction,0.655430,0.302836,0.041734
1078,neutral,0.102161,0.781847,0.115993
1079,entailment,0.000774,0.081088,0.918138


,first,second
0,You never give up.,You never find it difficult to change your min...
1,You never give up.,"Whatever you have set your sights on, you refu..."
2,You never give up.,You are patient unless someone takes you too far.
3,You never give up.,You are usually slow to anger unless someone t...
4,You never give up.,You are reliable.
...,...,...
1076,You are sensitive.,You are gentle.
1077,You are sensitive.,You are romantic.
1078,You are sentimental.,You are gentle.
1079,You are sentimental.,You are romantic.


In [5]:
def gen_dist(row):
    '''
    Generate distance score based on Contradiction, Entailment and Neutrality probabilities.
    Assumes that C+E+N=1.
    '''
    C = row['C']
    E = row['E']
    N = row['N']
    
    if C > 0.5:
        return C
    elif C == E:
        return 0.5
    else:
        return N/(N+E)*0.5

In [6]:
ser_res = df.apply(lambda row: gen_dist(row), axis=1)
ser_res.name = 'dist'
df = df.join(ser_res)
df.head()

,predicted_class,C,N,E,dist
0,neutral,0.022798,0.880442,0.096760,0.450491
1,entailment,0.001746,0.039801,0.958454,0.019935
2,neutral,0.004882,0.990273,0.004845,0.497565
3,neutral,0.052050,0.944104,0.003846,0.497971
4,entailment,0.001493,0.161846,0.836661,0.081044


In [7]:
save_path = input("enter path to save csv file of pairwise distance scores:\n")
df.to_csv(save_path)

enter path to save csv file of pairwise distance scores:
 ../data/roberta/Astro1_decomposed_clean_pairs_roberta_distances.csv
